# **Predicting High Volatility from ATR**

仮説：ボラティリティクラスタリングがあるなら、ATRとかのバーの幅をとるやつを計算して、その幅を超えたら、次の時間幅でも同様に超えるんとちゃうんか？=ボラがATRで予測できる？

In [2]:
import ccxt
import pandas as pd
hyperliquid = ccxt.hyperliquid()

### **実験方法** <br>
1. ボラティリティ（幅）をまずは各データに対して計算。
2. ATRの計算
3. ATRの幅 < その時間の幅 なら、次の時間の幅とATRを比較。（結果はT/Fの二値に分類されるはず）
4. 超えてるところの割合を調べる。
<br>

### **使用データ**
Hyperliquid, ccxtから得られるOHLCV.

### **OHLCVのダウンロード**

In [40]:
symbol_perp = 'BTC/USDC:USDC'
timeframe    = '1m'
ohlcv = hyperliquid.fetch_ohlcv(symbol_perp, timeframe, limit = 5000 )
df = pd.DataFrame(ohlcv, columns = ['timestamp','open','high','low','close','volume'])
df.head()

,timestamp,open,high,low,close,volume
0,1745394600000,93757.0,93759.0,93746.0,93747.0,0.89816
1,1745394660000,93746.0,93766.0,93746.0,93747.0,1.37882
2,1745394720000,93746.0,93821.0,93746.0,93818.0,2.24451
3,1745394780000,93817.0,93844.0,93817.0,93837.0,4.63537
4,1745394840000,93836.0,93890.0,93836.0,93875.0,23.66838


#### **各時間帯のレンジを計算**

In [41]:
df['range_bar'] = df['open'] - df['close'] #実態の足のレンジ計算
df['range_whisker'] = df['high'] - df['low'] #髭の計算。あってる？これ

In [42]:
df['abs_range_bar'] = abs(df['range_bar'])
df['atr_like'] = df['abs_range_bar'].rolling(window=50).mean() #過去50分のバーの幅の平均
df.dropna(inplace=True)

ここで一旦、<br>
1. バーの幅、その絶対値
2. 髭の幅
3. 過去50分のバーの幅（絶対値）の平均が取れた。

超えてるかどうかを判定する

In [43]:
df['is_high_vol'] = (df['abs_range_bar'] > df['atr_like']).astype(int) #その時間のバーの幅がATR_likeを超えてたら1, 超えてないと0

In [44]:
df['is_high_vol_next'] = df['is_high_vol'].shift(-1) #1こ未来のを今のところに下ろしてる（時間軸を1つ前に持ってきてる）
df.dropna(inplace=True) 

超えてたかどうか、の部分

In [46]:
df.head()

,timestamp,open,high,low,close,volume,range_bar,range_whisker,abs_range_bar,atr_like,is_high_vol,is_high_vol_next
49,1745397540000,94205.0,94205.0,94199.0,94202.0,7.72317,3.0,6.0,3.0,52.22,0,0.0
50,1745397600000,94201.0,94282.0,94201.0,94211.0,37.44831,-10.0,81.0,10.0,52.22,0,0.0
51,1745397660000,94211.0,94235.0,94208.0,94227.0,5.35928,-16.0,27.0,16.0,52.52,0,0.0
52,1745397720000,94227.0,94245.0,94173.0,94198.0,16.44716,29.0,72.0,29.0,51.66,0,0.0
53,1745397780000,94181.0,94198.0,94167.0,94184.0,35.33901,-3.0,31.0,3.0,51.32,0,1.0


In [47]:
next_high_vol_rate = df.loc[df['is_high_vol'] == 1, 'is_high_vol_next'].mean()#is_high_volが1のところを探して、is_high_vol_nextの平均を探してる。

In [48]:
next_high_vol_rate 

np.float64(0.42345110087045573)

## **結論**
一旦、過去のバーの50個の平均を超えると次も結構な確率（42.5%)で超えてることがわかった。<br>
### **他にできること？**
1. この42.5%という数字が大きいのか？小さいのか？
2. rollingは大きいほうがいい？小さい方がいい？
3. フラグ立てる閾値を厳しくするとどうか？ex)ATR_like * 1.2
4. Range_Whisker を使った場合どうか？
5. 2回連続で超えた場合は？

## **1. 42.5%という数字が良いのか？悪いのか？**
まずは、条件なしの高ボラ率を出してみる。<br>


In [49]:
base_rate = df['is_high_vol'].mean()
print(base_rate)

0.39454545454545453


ベースの高ボラ率は39.6%と、かなり高い。<br>
つまり、<br>
1. ボラティリティクラスタリングはあるか？: うっすらありそう。
2. 超強い現象なのか？: そこまで。ここでは一旦3%のほどの違いしか出なかった。
3. 戦略にはまだ使えなさそう。

## **2. rollingの窓を変えてみるとどうなるのか？**

1つ目の実験では、rolling = 50とした。これを20, 100などと長めにとった場合どうなるのかを確認する。

In [50]:
df.head()

,timestamp,open,high,low,close,volume,range_bar,range_whisker,abs_range_bar,atr_like,is_high_vol,is_high_vol_next
49,1745397540000,94205.0,94205.0,94199.0,94202.0,7.72317,3.0,6.0,3.0,52.22,0,0.0
50,1745397600000,94201.0,94282.0,94201.0,94211.0,37.44831,-10.0,81.0,10.0,52.22,0,0.0
51,1745397660000,94211.0,94235.0,94208.0,94227.0,5.35928,-16.0,27.0,16.0,52.52,0,0.0
52,1745397720000,94227.0,94245.0,94173.0,94198.0,16.44716,29.0,72.0,29.0,51.66,0,0.0
53,1745397780000,94181.0,94198.0,94167.0,94184.0,35.33901,-3.0,31.0,3.0,51.32,0,1.0


In [51]:
df['atr_like_20'] = df['abs_range_bar'].rolling(window=20).mean()
df['atr_like_100'] = df['abs_range_bar'].rolling(window=100).mean()
df.dropna(inplace=True)
df['is_high_vol_20'] = (df['abs_range_bar'] > df['atr_like_20']).astype(int)
df['is_high_vol_100'] = (df['abs_range_bar'] > df['atr_like_100']).astype(int)

df['is_high_vol_next_20'] = df['is_high_vol_20'].shift(-1)
df['is_high_vol_next_100'] = df['is_high_vol_100'].shift(-1)
df.dropna(inplace=True)

next_high_vol_rate_20 = df.loc[df['is_high_vol_20'] == 1, 'is_high_vol_next_20'].mean()
next_high_vol_rate_100 = df.loc[df['is_high_vol_100'] == 1, 'is_high_vol_next_100'].mean()

base_rate_20 = df['is_high_vol_20'].mean()
base_rate_100 = df['is_high_vol_100'].mean()

print(f"--- Rolling 20本 ---")
print(f"ベース確率: {base_rate_20:.4f} （{base_rate_20 * 100:.2f}%）")
print(f"超えた後の次も高ボラ確率: {next_high_vol_rate_20:.4f} （{next_high_vol_rate_20 * 100:.2f}%）\n")

print(f"--- Rolling 100本 ---")
print(f"ベース確率: {base_rate_100:.4f} （{base_rate_100 * 100:.2f}%）")
print(f"超えた後の次も高ボラ確率: {next_high_vol_rate_100:.4f} （{next_high_vol_rate_100 * 100:.2f}%）")


--- Rolling 20本 ---
ベース確率: 0.3971 （39.71%）
超えた後の次も高ボラ確率: 0.4356 （43.56%）

--- Rolling 100本 ---
ベース確率: 0.3885 （38.85%）
超えた後の次も高ボラ確率: 0.4278 （42.78%）


### Rollingの幅を変えたところでそれほど大きな差はなかった。<br>
あと気になるのは、ベースレートと比較してどうすんの？みたいな。投資の指標として使うんだから、ベースの確率と比較するというよりかはランダムに予測して合ってるかどうか、みたいな比較がフェアってもんじゃないのか？

# **2. 髭込みのボラティリティではどうなのか？**

In [55]:
df['atr_like_whisker'] = df['range_whisker'].rolling(window=50).mean()
df['is_high_vol_whisker'] = (df['range_whisker'] > df['atr_like_whisker']).astype(int) #高ボラフラグ作成
df['is_high_vol_next_whisker'] = df['is_high_vol_whisker'].shift(-1)
df.dropna(inplace=True)
next_high_vol_rate_whisker = df.loc[df['is_high_vol_whisker'] == 1, 'is_high_vol_next_whisker'].mean()
base_rate_whisker = df['is_high_vol_whisker'].mean()
print("--- 髭込みボラ版 ---")
print(f"ベース確率: {base_rate_whisker:.4f} ({base_rate_whisker * 100:.2f}%)")
print(f"超えた後の次も高ボラ確率: {next_high_vol_rate_whisker:.4f} ({next_high_vol_rate_whisker * 100:.2f}%)")

--- 髭込みボラ版 ---
ベース確率: 0.3977 (39.77%)
超えた後の次も高ボラ確率: 0.4866 (48.66%)


お？

### **閾値をちょい高くしたバージョン**

In [56]:
threshold = 1.2  # ← ここを1.2倍にする
df['is_high_vol_whisker_strict'] = (df['range_whisker'] > threshold * df['atr_like_whisker']).astype(int)
df['is_high_vol_next_whisker_strict'] = df['is_high_vol_whisker_strict'].shift(-1)

next_high_vol_rate_whisker_strict = df.loc[df['is_high_vol_whisker_strict'] == 1, 'is_high_vol_next_whisker_strict'].mean()
base_rate_whisker_strict = df['is_high_vol_whisker_strict'].mean()

print("--- 髭込みボラ版（しきい値1.2倍） ---")
print(f"ベース確率: {base_rate_whisker_strict:.4f} ({base_rate_whisker_strict * 100:.2f}%)")
print(f"超えた後の次も高ボラ確率: {next_high_vol_rate_whisker_strict:.4f} ({next_high_vol_rate_whisker_strict * 100:.2f}%)")

--- 髭込みボラ版（しきい値1.2倍） ---
ベース確率: 0.2935 (29.35%)
超えた後の次も高ボラ確率: 0.3956 (39.56%)


In [59]:
# まず、次の足のフラグもすでにshiftしてあるので、さらに次のフラグを作る
df['is_high_vol_next2_whisker'] = df['is_high_vol_whisker'].shift(-2)

# 2本連続で爆発してるかをチェック
two_consecutive_explosions = (df['is_high_vol_whisker'] == 1) & (df['is_high_vol_next_whisker'] == 1)

# 2本連続爆発の割合を計算
two_consecutive_explosion_rate = two_consecutive_explosions.mean()

print("--- 2本連続爆発版（しきい値1倍） ---")
print(f"2本連続爆発する確率: {two_consecutive_explosion_rate:.4f} ({two_consecutive_explosion_rate * 100:.2f}%)")


--- 2本連続爆発版（しきい値1倍） ---
2本連続爆発する確率: 0.1935 (19.35%)


In [61]:
# もう1回貼っとく
df['close_next'] = df['close'].shift(-1)
df['return_after_explosion'] = df['close_next'] - df['close']

df_explosion = df[df['is_high_vol_whisker'] == 1]

mean_return_after_explosion = df_explosion['return_after_explosion'].mean()

print("--- 爆発直後の方向性 ---")
print(f"爆発直後のリターン平均: {mean_return_after_explosion:.6f}")

--- 爆発直後の方向性 ---
爆発直後のリターン平均: 1.895702


検証テーマ	結果・考察 <br>
abs(open-close)ベースでボラを見る	クラスタリングは弱め（ベース39.6%、次ボラ42.5%）<br>
high-low（髭込み）ベースでボラを見る	クラスタリングやや強め（ベース39.4%、次ボラ48.7%）<br>
しきい値を1.2倍にすると？	爆発バーのベース確率下がるが、次ボラ連鎖率は保たれる（39.6%）<br>
2本連続爆発するか？	1.0倍基準なら19%、1.2倍基準なら11%程度（減少）<br>
爆発直後の方向性は？	爆発直後、平均+1.9ドルのリターンあり → 順張り期待値プラス<br>

🔹 分かったこと
髭込みボラを使ったほうが連鎖傾向が強く出る <br>
爆発後に「順張り」でエントリーする発想はアリ <br>
ただし2本連続で爆発するケースはそこまで多くない <br>
爆発を検知したら、次の1分に張る、という超短期スキャル戦略には現実味あり <br>

# **次は？**